In [13]:
import numpy as np
import pandas as pd

import algo
import federated

import os
import json

rand_seed = 42

np.random.seed(rand_seed)

x_target_train = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
y_target_train = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
x_target_test = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
y_target_test = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
n_classes = len(np.unique(y_target_train))

number_of_clients = 4
fl_iterations = 1
data_per_client = int(x_target_train.shape[0]/number_of_clients)

for m_it in [5]:
    #create clients with set training parameters and datasets
    clients = {}
    for i in range(number_of_clients):
        clients[i] = algo.LogisticRegression_DPSGD()

        clients[i].n_classes      = n_classes
        clients[i].alpha          = 0.001
        clients[i].max_iter       = m_it
        clients[i].lambda_        = 1e-5
        clients[i].tolerance      = 1e-5
        clients[i].DP             = False
        clients[i].L              = 10
        clients[i].epsilon        = 0

        params = dict(clients[0].__dict__)

        clients[i].x = x_target_train[i*data_per_client:(i+1)*data_per_client]
        clients[i].y = y_target_train[i*data_per_client:(i+1)*data_per_client]

    fl_path = f'fl/rs{rand_seed}_ncl{number_of_clients}_fiter{fl_iterations}_lr{clients[0].alpha}_iter{clients[0].max_iter}_reg{clients[0].lambda_}_DP{clients[0].DP}'
    if clients[0].DP:
        fl_path += f'_eps{clients[0].epsilon}_L{clients[0].L}'


    params.pop('x')
    params.pop('y')
    print(params)
    if os.path.exists(fl_path):

        print('Experiment already exists')

    else:

        print('Creating new folder:\n', fl_path)
        os.mkdir(fl_path)
        with open(fl_path+'/params.json', 'w') as file:
            json.dump(params, file)
        results = {}
        for iteration in range(fl_iterations):

            print(iteration, ' FL iteration')
            for i in clients:

                print("Start training client: ", i)
                federated.train_client(clients[i], x_target_test, y_target_test)
                np.save(fl_path + f'/i{iteration}_c{i}', clients[i].theta)
                results[f'i{iteration}_c{i}'] = (clients[i].train_acc,  clients[i].test_acc)


            global_model = federated.aggregate(clients)
            np.save(fl_path + f'/i{iteration}_g', global_model)
            federated.update_clients(clients, global_model)
            #global model evaluation
            print('Global model evaluataion:')
            gtrain_acc = clients[0].evaluate(x_target_train, y_target_train, acc=True)
            gtest_acc = clients[0].evaluate(x_target_test, y_target_test, acc=True)
            results[f'i{iteration}_g'] = (gtrain_acc,  gtest_acc)


    res = pd.DataFrame.from_dict(results, orient='index')
    res.to_csv(fl_path + f'/results.csv')

{'n_classes': 100, 'alpha': 0.001, 'max_iter': 5, 'lambda_': 1e-05, 'tolerance': 1e-05, 'DP': False, 'L': 10, 'C': 1, 'epsilon': 0, 'delta': 1e-05}
Creating new folder:
 fl/rs42_ncl4_fiter1_lr0.001_iter5_reg1e-05_DPFalse
0  FL iteration
Start training client:  0
The accuracy of the model : 4.8 %
The accuracy of the model : 3.6999999999999997 %
Start training client:  1
The accuracy of the model : 3.2 %
The accuracy of the model : 2.5 %
Start training client:  2
The accuracy of the model : 3.0 %
The accuracy of the model : 2.8000000000000003 %
Start training client:  3
The accuracy of the model : 3.2 %
The accuracy of the model : 2.1999999999999997 %
Global model evaluataion:
The accuracy of the model : 3.5000000000000004 %
The accuracy of the model : 2.7 %


In [14]:
path = 'fl'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:
            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            for k in results[r]['Unnamed: 0']:
                it.append(k[k.find('i')+1:k.find('_')])
                client.append(k[k.find('_')+1:])
            results[r]['it'] = it
            results[r]['client'] = client
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
            models[r][file] = np.load(r+'/'+file)



In [4]:
x_target_train = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
y_target_train = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
x_target_test = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
y_target_test = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
n_classes = len(np.unique(y_target_train))


In [23]:
def aggregate(clients):
    
    global_model = clients[0].theta 
    for i in range(1, len(clients.keys())):
        global_model += clients[i].theta 

    return global_model/3

In [24]:
def update_clients(clients, global_model):
    
    for i in clients:
        clients[i].theta = global_model

In [25]:
def train_client(client):
    
    X,y = client.init_theta(client.x, client.y)
    client.train(X,y)
    client.train_acc = client.evaluate(client.x, client.y, acc=True)
    client.test_acc = client.evaluate(x_target_test, y_target_test, acc=True)


In [35]:
   

number_of_clients = 2
fl_iterations = 5
data_per_client = int(x_target_train.shape[0]/number_of_clients)

for m_it in [1]:
    #create clients with set training parameters and datasets
    clients = {}
    for i in range(number_of_clients):
        clients[i] = algo.LogisticRegression_DPSGD()

        clients[i].n_classes      = n_classes
        clients[i].alpha          = 0.001
        clients[i].max_iter       = m_it
        clients[i].lambda_        = 1e-5
        clients[i].tolerance      = 1e-5
        clients[i].DP             = False
        clients[i].L              = 10
        clients[i].epsilon        = 0

        params = dict(clients[0].__dict__)

        clients[i].x = x_target_train[i*data_per_client:(i+1)*data_per_client]
        clients[i].y = y_target_train[i*data_per_client:(i+1)*data_per_client]

    fl_path = f'fl/rs{rand_seed}_ncl{number_of_clients}_fiter{fl_iterations}_lr{clients[0].alpha}_iter{clients[0].max_iter}_reg{clients[0].lambda_}_DP{clients[0].DP}'
    if clients[0].DP:
        fl_path += f'_eps{clients[0].epsilon}_L{clients[0].L}'


    params.pop('x')
    params.pop('y')
    print(params)
    if os.path.exists(fl_path):

        print('Experiment already exists')

    else:

        print('Creating new folder:\n', fl_path)
        os.mkdir(fl_path)
        with open(fl_path+'/params.json', 'w') as file:
            json.dump(params, file)
        results = {}
        for iteration in range(fl_iterations):

            print(iteration, ' FL iteration')
            for i in clients:

                print("Start training client: ", i)
                train_client(clients[i], x_target_test, y_target_test)
                np.save(fl_path + f'/i{iteration}_c{i}', clients[i].theta)
                results[f'i{iteration}_c{i}'] = (clients[i].train_acc,  clients[i].test_acc)


            global_model = aggregate(clients)
            np.save(fl_path + f'/i{iteration}_g', global_model)
            update_clients(clients, global_model)
            #global model evaluation
            print('Global model evaluataion:')
            gtrain_acc = clients[0].evaluate(x_target_train, y_target_train, acc=True)
            gtest_acc = clients[0].evaluate(x_target_test, y_target_test, acc=True)
            results[f'i{iteration}_g'] = (gtrain_acc,  gtest_acc)


    res = pd.DataFrame.from_dict(results, orient='index')
    res.to_csv(fl_path + f'/results.csv')

{'n_classes': 100, 'alpha': 0.001, 'max_iter': 1, 'lambda_': 1e-05, 'tolerance': 1e-05, 'DP': False, 'L': 10, 'C': 1, 'epsilon': 0, 'delta': 1e-05}
Creating new folder:
 fl/rs42_ncl2_fiter5_lr0.001_iter1_reg1e-05_DPFalse
0  FL iteration
Start training client:  0
The accuracy of the model : 3.0 %
The accuracy of the model : 2.1999999999999997 %
Start training client:  1
The accuracy of the model : 2.8000000000000003 %
The accuracy of the model : 2.1999999999999997 %
Global model evaluataion:
The accuracy of the model : 2.9000000000000004 %
The accuracy of the model : 2.1999999999999997 %
1  FL iteration
Start training client:  0
The accuracy of the model : 5.7 %
The accuracy of the model : 4.5 %
Start training client:  1
The accuracy of the model : 2.8000000000000003 %
The accuracy of the model : 2.1999999999999997 %
Global model evaluataion:
The accuracy of the model : 2.9000000000000004 %
The accuracy of the model : 2.1999999999999997 %
2  FL iteration
Start training client:  0
The ac

'fl/rs42_ncl2_fiter5_lr0.001_iter100_reg1e-05_DPFalse/results'